In [ ]:
from astropy.coordinates import SkyCoord

NAMES = [
    ["CRTS J062029.9-250235", "Gaia DR3 2912290471563097088", "ATO J095.1241-25.0432"],
    ["CRTS J035520.3-484728", "Gaia DR3 4831519899384690944", "ATO J058.8350-48.7912"],
    ["Gaia DR3 2912314003686113664",],
    ["Gaia DR3 2900262462891972608",],
    ["CRTS J061939.7-244148", "Gaia DR3 2912315481154886016", "ATO J094.9162-24.6965"],
]
coords = SkyCoord([SkyCoord.from_name(names[0]) for names in NAMES])
coords
RADEC = list(zip(coords.ra.deg, coords.dec.deg))

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.time import Time

from approx_light_travel_time import fast_light_travel_time_heliocentric_elliptical

def helio_mjd(mjd, ra, dec):
    coord = SkyCoord(ra=ra, dec=dec, unit="deg")
    time = Time(mjd, format="mjd", scale="tai")
    helio_time = time + fast_light_travel_time_heliocentric_elliptical(time, coord)
    return helio_time.mjd

def add_helio_mjd(df):
    df["diaObjectForcedSource.helioMjd"] = helio_mjd(
        df["diaObjectForcedSource.midpointMjdTai"],
        df["diaObjectForcedSource.coord_ra"],
        df["diaObjectForcedSource.coord_dec"],
    )
    return df

In [ ]:
GAIA_BANDS = ["G", "BP", "RP"]
CRTS_BANDS = ["V"]
ATLAS_BANDS = ["c", "o"]
OTHER_BANDS = GAIA_BANDS + CRTS_BANDS + ATLAS_BANDS

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.time import Time, TimeDelta
from astroquery.gaia import Gaia

def load_crts(name):
    path = f"{name}.csv"
    df = pd.read_csv(
        path,
    ).rename(
        columns=str.lower,
    ).assign(
        band="V",
    )
    df["helioMjd"] = helio_mjd(df["mjd"], df["ra"], df["dec"])
    return df


def load_atlas(name):
    path = f"{name}.txt"
    table = Table.read(path, format="ascii")
    df = table.to_pandas()
    df = df.query("err == 0")
    df = df.rename(columns={"m": "mag", "dm": "magerr", "F": "band", "##MJD": "mjd", "RA": "ra", "Dec": "dec"})
    df["helioMjd"] = helio_mjd(df["mjd"], df["ra"], df["dec"])
    # https://fallingstar-data.com/forcedphot/faq/
    df = df.query(
        "duJy < 10000"
        " and err == 0"
        " and x > 100"
        " and x < 10460"
        " and y > 100"
        " and y < 10460"
        " and maj < 5"
        " and maj > 1.6"
        " and min < 5"
        " and min > 1.6"
        " and apfit > -1"
        " and apfit < -0.1"
        " and mag5sig > 17"
        " and Sky > 17"
        # addition
        " and 10 < mag < 20"
    )
    return df


def load_gaia(name):
    """Adopted from
    https://github.com/snad-space/ztf-viewer/blob/7730255f7e0a8da189a2a88b22eceda81b0a1298/ztf_viewer/catalogs/conesearch/gaia_dr3.py
    """
    path = Path(f"{name}.parquet")
    if path.exists():
        return pd.read_parquet(path)

    # https://www.cosmos.esa.int/web/gaia/edr3-passbands
    AB_ZP = {
        "G": 25.8010446445,
        "BP": 25.3539555559,
        "RP": 25.1039837393,
    }

    id = name.removeprefix("Gaia DR3 ")
    results = Gaia.load_data(
        ids=[id],
        data_release="Gaia DR3",
        retrieval_type="EPOCH_PHOTOMETRY",
        data_structure="INDIVIDUAL",
    )
    assert len(results) == 1
    tables = next(iter(results.values()))
    assert len(tables) == 1
    table = tables[0].to_table()  # From VOtable to normal astropy table
    table = table[~table["rejected_by_photometry"]]
    
    band_tables = {band: table[~table[f"variability_flag_{band.lower()}_reject"]] for band in GAIA_BANDS}
    band = np.concatenate([np.full(len(tab), band) for band, tab in band_tables.items()])
    gaia_time = np.concatenate(
            [band_tables["G"]["g_transit_time"]] + [band_tables[band][f"{band.lower()}_obs_time"] for band in ["BP", "RP"]]
        )
    tcb = Time("2010-01-01T00:00:00", scale="tcb") + TimeDelta(gaia_time, format="jd")
    helioMjd = Time(tcb, scale="tai").mjd

    ab_zp = np.vectorize(AB_ZP.get)(band)
    gaia_flux = np.concatenate(
        [band_tables["G"]["g_transit_flux"]] + [band_tables[band][f"{band.lower()}_flux"] for band in ["BP", "RP"]]
    )
    mag = ab_zp - 2.5 * np.log10(gaia_flux)

    flux_over_error = np.concatenate(
        [band_tables["G"]["g_transit_flux_over_error"]]
        + [band_tables[band][f"{band.lower()}_flux_over_error"] for band in ["BP", "RP"]]
    )
    magerr = 2.5 / np.log(10.0) / flux_over_error

    df = pd.DataFrame({
        "mag": mag,
        "magerr": magerr,
        "helioMjd": helioMjd,
        "band": band,
    })
    df.to_parquet(path)
    return df


def load(names):
    dfs = []
    for name in names:
        if name.startswith("CRTS"):
            dfs.append(load_crts(name))
        elif name.startswith("Gaia"):
            dfs.append(load_gaia(name))
        elif name.startswith("ATO"):
            dfs.append(load_atlas(name))
        else:
            raise ValueError(f"Uknown name: {name}")
    return pd.concat(dfs, axis=0)

OTHER = list(map(load, NAMES))

for df in OTHER:
    display(df)

In [ ]:
import lsdb

rows = []
for ra, dec in RADEC:
    cat = lsdb.open_catalog(
        "/sdf/data/rubin/user/kostya/hats/dia_object_lc_15k",
        search_filter=lsdb.ConeSearch(ra, dec, radius_arcsec=0.5),
    ).map_partitions(add_helio_mjd)
    df = cat.compute()
    assert len(df) == 1, f"{len(df) = }, {ra = }, {dec = }"
    row = df.iloc[0]
    rows.append(row)
rows

In [ ]:
lcs = [row["diaObjectForcedSource"].query(
    "~psfFlux_flag"
    " and ~psfDiffFlux_flag"
    " and ~pixelFlags_suspect"
    " and ~pixelFlags_saturated"
    " and ~pixelFlags_cr"
    " and ~pixelFlags_bad"
) for row in rows]
display(lcs[0])
display(lcs[1])

In [ ]:
from pathlib import Path

import numpy as np
from astropy.timeseries import LombScargleMultiband
from dask.distributed import Client
from tqdm import tqdm

PERIODS = [0.235668, 0.565363, 0.241915000000, 0.245037000000, 0.240780]

def extract_period(lc, other, period0):
    minimum_frequency = 1.0 / 0.6
    maximum_frequency = 1.0 / 0.2
    if period0 is not None:
        minimum_frequency = 0.9999/period0
        maximum_frequency = 1.0001/period0
    
    # other_dy_factor = 0.1 # increase weights of points from another survey
    
    bands = np.concat([lc["band"], other["band"]])
    t = np.concat([lc["helioMjd"], other["helioMjd"]])
    y = np.concat([lc["psfMag"], other["mag"]])
    # dy = np.concat([lc["psfMagErr"], other["magerr"] * other_dy_factor])
    dy = 1.0
    model = LombScargleMultiband(bands=bands, t=t, y=y, dy=dy, nterms_band=1, nterms_base=3)
    freq = np.linspace(minimum_frequency, maximum_frequency, 10_000)
    power = model.power(freq)
    return 1.0 / freq[np.argmax(power)]

periods_path = Path('periods.npy')
if periods_path.exists():
    periods = np.load(periods_path).tolist()
    periods = [p1 if p2 is None else p2 for p1, p2 in zip(periods, PERIODS)]
else:
    futures = []
    with Client(n_workers=len(NAMES)) as client:
        display(client)
        for args in zip(lcs, OTHER, PERIODS):
            futures.append(client.submit(extract_period, *args))
        periods = [f.result() for f in tqdm(futures)]
    np.save(periods_path, periods)
periods

In [ ]:
# !mv periods.npy periods_orig.npy
# !rm periods.npy

In [ ]:
import matplotlib.pyplot as plt

COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000',
          "G": "grey", "BP": "#a4d1e3", "RP": "#e6e68f",
          'V': "#ff6a06",
          "c": "cyan", "o": "orange"}

for names, row, lc, period, other in zip(NAMES, rows, lcs, periods, OTHER):
    name = names[0]

    minimum_time = lc["helioMjd"][lc.query("band == 'r'")["psfMag"].idxmax()]
    lc["phase"] = (lc["helioMjd"] - minimum_time) % period / period

    plt.figure()
    for band in "ugrizy":
        data = lc.query("band == @band")
        label = band
        offset = 0.0
        if band == 'u':
            offset = -1.0
            label = f"{label}${offset:+.0f}$"
        plt.errorbar(data["phase"], data["psfMag"] + offset, yerr=data["psfMagErr"], label=label, fmt='o', color=COLORS[band])
    
    if other is not None:
        other = other.copy()
        other["phase"] = (other["helioMjd"] - minimum_time) % period / period
        for band in OTHER_BANDS:
            data = other.query("band == @band")
            if len(data) == 0:
                continue
            if band == "V":
                label = f"CRTS {band}"
                alpha = 0.15
                fmt = "s"
            elif band in GAIA_BANDS:
                label = f"Gaia {band}"
                alpha = 1.0
                fmt = "^"
            else:
                label = f"ATLAS {band}"
                alpha = 0.05
                fmt = "*"
            plt.errorbar(data["phase"], data["mag"], yerr=data["magerr"], label=label, fmt=fmt, color=COLORS[band], alpha=alpha, zorder=-1)

    plt.gca().invert_yaxis()
    ylim = [None, None]
    if name == 'CRTS J062029.9-250235':
        ylim = [16.2, 15.5]
    elif name == "CRTS J035520.3-484728":
        ylim = [18.7, 17.0]
    elif name == "Gaia DR3 2900262462891972608":
        ylim = [22.5, None]
    elif name == "CRTS J061939.7-244148":
        ylim = [18.3, 16.2]
    plt.xlim([0, 1])
    plt.ylim(ylim)
    plt.xlabel("phase")
    plt.ylabel("PSF mag")
    plt.title(f"{name}\ndiaObjectID {row.diaObjectId}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$, P={period:.7f} d")
    plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
    plt.savefig(f"{row.diaObjectId}.pdf", bbox_inches='tight')